In [1]:
import pandas as pd
import requests

# URL for the INSEE commune data
url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"

# Load the CSV file containing the list of communes
df = pd.read_csv(url)
print("File successfully loaded!")
print(df.head())

# Set the Georisques API URL
georisques_api_url = "https://georisques.gouv.fr/api/v1/gaspar/tri"

# Function to call the API and get flood risk data for a specific commune
def get_flood_risk_data(commune_code):
    request_url = f"{georisques_api_url}?code_insee={commune_code}&page=1&page_size=10"
    response = requests.get(request_url)
    if response.status_code == 200:
        json_data = response.json()
        # Check if the 'data' key exists and is non-empty
        if json_data and 'data' in json_data and len(json_data['data']) > 0:
            # Convert the relevant data to a DataFrame
            risk_data = pd.DataFrame(json_data['data'])
            return risk_data
    return None  # Return None for communes without valid flood risk data

# Create an empty list to store all the flood risk data
all_risk_data = []

# Loop through the DataFrame and fetch flood risk data for each commune
for index, row in df.iterrows():
    commune_code = row['COM']  # Assuming 'COM' is the column containing the commune code
    risk_data = get_flood_risk_data(commune_code)
    if risk_data is not None:
        # Append the DataFrame for this commune to the list
        all_risk_data.append(risk_data)

# Concatenate all the individual DataFrames into a single DataFrame
if all_risk_data:
    combined_risk_data = pd.concat(all_risk_data, ignore_index=True)
    print("Combined flood risk data:")
    print(combined_risk_data.head())
else:
    print("No valid flood risk data found for any commune.")

# Save the combined data to a CSV file
combined_risk_data.to_csv("flood_risk_data.csv", index=False)
print("Flood risk data saved to 'flood_risk_data.csv'")


File successfully loaded!
  TYPECOM    COM   REG DEP CTCD  ARR  TNCC                    NCC  \
0     COM  01001  84.0  01  01D  012     5  ABERGEMENT CLEMENCIAT   
1     COM  01002  84.0  01  01D  011     5    ABERGEMENT DE VAREY   
2     COM  01004  84.0  01  01D  011     1      AMBERIEU EN BUGEY   
3     COM  01005  84.0  01  01D  012     1    AMBERIEUX EN DOMBES   
4     COM  01006  84.0  01  01D  011     1                AMBLEON   

                  NCCENR                  LIBELLE   CAN  COMPARENT  
0  Abergement-Clémenciat  L'Abergement-Clémenciat  0108        NaN  
1    Abergement-de-Varey    L'Abergement-de-Varey  0101        NaN  
2      Ambérieu-en-Bugey        Ambérieu-en-Bugey  0101        NaN  
3    Ambérieux-en-Dombes      Ambérieux-en-Dombes  0122        NaN  
4                Ambléon                  Ambléon  0104        NaN  
Combined flood risk data:
  code_national_tri       libelle_tri  \
0   01DREAL20160001              Lyon   
1   01DREAL20160001              Lyon